# Trabalho 2

11218813 Bruna Magrini da Cruz  
11218809 Flavio Salles  
11219004 Yasmin Osajima de Araujo  

## Bibliotecas

In [ ]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image
import random

## Janela

In [ ]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1200
largura = 1600
window = glfw.create_window(largura, altura, "Trabalho 2 - Jardim do ICMC", None, None)
glfw.make_context_current(window)

## GLSL 

Aqui, utilizamos o GLSL para definir o Vertex Shader (processamento dos vértices individuais) e Fragment Shader (processamento das cores dos pixels).

In [ ]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(position);
            out_normal = normals;
        }
        """

fragment_code = """
        // parametro com a cor da(s) fonte(s) de iluminacao
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        vec3 lightColor = vec3(1.0, 1.0, 1.0);
        
        // parametros da iluminacao ambiente e difusa
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        


        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * lightColor;             
        
            // calculando reflexao difusa
            vec3 norm = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir = normalize(lightPos - out_fragPos); // direcao da luz
            float diff = max(dot(norm, lightDir), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse = kd * diff * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir = normalize(reflect(-lightDir, norm)); // direcao da reflexao
            float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
            vec3 specular = ks * spec * lightColor;             
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

## Compilação e Linkagem

Requisitando slot para a GPU para os programas Vertex Shader e Fragment Shader, associando o código-fonte aos slots solicitados e compilando ambos shaders.

In [ ]:
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")
    
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

Em seguida, associando os programas compilados ao programa principal e linkando o programa.

In [ ]:
glAttachShader(program, vertex)
glAttachShader(program, fragment)

glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
glUseProgram(program)

## Definição de Elementos

Nesse momento, compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

In [ ]:
MODELS_PATH = 'models/'
models_definition = {
    'terreno': { 'texture_id': 1, 'obj': 'terreno/terreno2.obj', 'texture':'terreno/grama.jpg' },
    'bola': { 'texture_id': 2, 'obj': 'ball/ball.obj', 'texture': 'ball/10536_soccerball_V1.jpg' },
    'banco': { 'texture_id': 3, 'obj': 'bench/bench.obj', 'texture': 'bench/ConcreteBench.jpg' },
    'arvore': { 'texture_id': 4, 'obj': 'tree/tree.obj', 'texture': 'tree/Kabak_Bark_and_Leaves_v1_S.png' },
    'luz': { 'texture_id': 5, 'obj': 'luz/luz.obj', 'texture':  'luz/luz.png'},
    'grama': { 'texture_id': 6, 'obj': 'terreno/grama.obj', 'texture':'terreno/montanha.png' },
    #'nuvem': { 'texture_id': 6, 'obj': 'nuvem/nuvem.obj', 'texture':  ''}
}

### Funções para leitura do wavefront e textura

In [ ]:
def load_model_obj(filename):
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(MODELS_PATH + filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando normais
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

In [ ]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(MODELS_PATH + img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### Carregando modelos (vértices e texturas)

In [ ]:
# Armazena todos os vertices carregados dos arquivos
vertices_list = [] 
normals_list = []  
textures_coord_list = []

In [ ]:
for model_name in models_definition:
    model_definition = models_definition[model_name]
    print("loading model: ", model_name)
    model = load_model_obj(model_definition['obj'])

    model_definition['vertices_start'] = len(vertices_list)

    # Inserindo vértices do modelo no vetor de vértices final
    for face in model['faces']:
        for vertice_id in face[0]:
            vertices_list.append( model['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( model['texture'][texture_id-1] )
        for normal_id in face[2]:
            if(model['normals'] == []): break
            normals_list.append( model['normals'][normal_id-1] )

    model_definition['vertices_size'] = len(vertices_list)-model_definition['vertices_start']

    # Carregando textura e definindo seu id único
    load_texture(model_definition['texture_id'], model_definition['texture'])

## Transmissão dos dados da CPU para a GPU

Precisamos requisitar dois slots:
- Um para enviar coordenadas dos vértices.
- Outro para enviar coordenadas de texturas.

In [ ]:
# Requisita um buffers na GPU
buffer = glGenBuffers(3)

# Envia coordenadas de vértices para a GPU
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

# Envia coordenadas de textura para a GPU
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

# Envia dados de iluminação para a GPU
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = normals_list

# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

## Matrizes Model, View e Projection

In [ ]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

## Desenhar modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo.
* É necessário indicar qual o ID da textura do modelo.

In [ ]:
#### define parametros de ilumincao do modelo
# ka coeficiente de reflexao ambiente do modelo
# kd coeficiente de reflexao difusa do modelo
# ks coeficiente de reflexao especular do modelo
# ns expoente de reflexao especular

def define_iluminacao(ka, kd, ks, ns):

    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ns pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu            


In [ ]:
def desenha_luz(t_X):
    # rotacao
    angle = 0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = t_X; t_y = 60.0; t_z = -100.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    define_iluminacao(1.0, 1.0, 1.0, 1000)

    loc_light_pos = glGetUniformLocation(program, "lightPos") # recuperando localizacao da variavel lightPos na GPU
    glUniform3f(loc_light_pos, t_x, t_y, t_z) ### posicao da fonte de luz

    model_definition = models_definition['luz']
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_definition['texture_id'])
    # Desenha o modelo
    glDrawArrays(GL_TRIANGLES, model_definition['vertices_start'], model_definition['vertices_size']) 

In [ ]:
def desenha_terreno():
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    #translacao
    t_x = 0.0; t_y = 0.0; t_z = 0.0;
    
    # escala
    s_x = 150; s_y = 1.0; s_z = 150;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    define_iluminacao(0.5, 0.1, 0.9, 5)

    model_definition = models_definition['terreno']
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_definition['texture_id'])
    # Desenha o modelo
    glDrawArrays(GL_TRIANGLES, model_definition['vertices_start'], model_definition['vertices_size']) 

In [ ]:
def desenha_grama(r_y, r_z):
    # rotacao
    angle = -90.0;
    r_x = 1.0;
    
    #translacao
    t_x = 0.0; t_y = -14.0; t_z = 0.0;
    
    # escala
    s_x = 0.01; s_y = 0.01; s_z = 0.01;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    define_iluminacao(0.5, 0.1, 0.9, 1)

    model_definition = models_definition['grama']
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_definition['texture_id'])
    # Desenha o modelo
    glDrawArrays(GL_TRIANGLES, model_definition['vertices_start'], model_definition['vertices_size']) 

In [ ]:
def desenha_bola(t_x, rotacao):
    # rotacao
    angle = rotacao_bola;
    r_x = 0.0; r_y = 0.0; r_z = 0.5;
    
    # translacao
    t_y = 1.0; t_z = 0.5;
    
    # escala
    s_x = 0.05; s_y = 0.05; s_z = 0.05;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    define_iluminacao(0.5, 0.1, 0.9, ns_inc)
    
    model_definition = models_definition['bola']
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_definition['texture_id'])
    # Desenha o modelo
    glDrawArrays(GL_QUADS, model_definition['vertices_start'], model_definition['vertices_size']) 

In [ ]:
def desenha_banco():
    # rotacao
    angle = -90.0;
    r_x = 1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = 0.0; t_z = 0.5;
    
    # escala
    s_x = 0.05; s_y = 0.05; s_z = 0.05;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    define_iluminacao(0.5, 0.1, 0.9, ns_inc)
    
    model_definition = models_definition['banco']
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_definition['texture_id'])
    # Desenha o modelo
    glDrawArrays(GL_QUADS, model_definition['vertices_start'], model_definition['vertices_size']) 

In [ ]:
def desenha_arvore():
    # rotacao
    angle = -90.0;
    r_x = 1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 10.0; t_y = 0.0; t_z = 0.5;
    
    # escala
    s_x = 0.3; s_y = 0.3; s_z = 0.3;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    define_iluminacao(0.5, 0.1, 0.9, ns_inc)
    
    model_definition = models_definition['arvore']
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_definition['texture_id'])
    # Desenha o modelo
    glDrawArrays(GL_TRIANGLES, model_definition['vertices_start'], model_definition['vertices_size']) 

## Eventos do Teclado

Para modificar a posição da câmera:
- Teclas A, S, D e W para movimentação no espaço tridimensional.
- Posição do mouse para "direcionar" a câmera.

In [ ]:
cameraPos   = glm.vec3(0.0,  5.0,  10.0);
cameraFront = glm.vec3(0.0,  0.0,  -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    global ns_inc
    
    cameraSpeed = 0.5
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
            
    if key == 265 and (action==1 or action==2): # tecla para cima
        ns_inc = ns_inc * 2
        
    if key == 264 and (action==1 or action==2): # tecla para baixo
        ns_inc = ns_inc / 2

    if cameraPos[0] < -100: cameraPos[0] = -100
    if cameraPos[0] > 100: cameraPos[0] = 100
    if cameraPos[1] < 1: cameraPos[1] = 1
    if cameraPos[1] > 40: cameraPos[1] = 40
    if cameraPos[2] < -100: cameraPos[2] = -100
    if cameraPos[2] > 100: cameraPos[2] = 100
            
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.2 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset
    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(math.radians(yaw)) * math.cos(math.radians(pitch))
    front.y = math.sin(math.radians(pitch))
    front.z = math.sin(math.radians(yaw)) * math.cos(math.radians(pitch))
    cameraFront = glm.normalize(front)

glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

## Execução

In [ ]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

In [ ]:
glEnable(GL_DEPTH_TEST)

x_bola = 0.0
direcao_bola = 1
rotacao_bola = 0
ns_inc = 100
x_luz = 0
incremento = 0.05

while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(0.5, 0.8, 0.92, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)

    desenha_terreno()
    desenha_grama(-0.1, 0)
    desenha_grama(0.1, 0)
    desenha_banco()
    desenha_arvore()
    desenha_luz(x_luz)
    
    x_luz += incremento
    if x_luz > 100 or x_luz < -100:
        incremento *= -1
    

    if x_bola > 5:
        direcao_bola = -1
    elif x_bola < -5:
        direcao_bola = 1
    x_bola = x_bola + direcao_bola * 0.05
    rotacao_bola = rotacao_bola+1
    desenha_bola(x_bola, rotacao_bola)

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)  

    # atualizando a posicao da camera/observador na GPU para calculo da reflexao especular
    loc_view_pos = glGetUniformLocation(program, "viewPos") # recuperando localizacao da variavel viewPos na GPU
    glUniform3f(loc_view_pos, cameraPos[0], cameraPos[1], cameraPos[2]) ### posicao da camera/observador (x,y,z)  
    
    glfw.swap_buffers(window)

glfw.terminate()